In [ ]:
#!/usr/bin/env python
# coding: utf-8

###general imports
import copy
import os
import shutil

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score
import keras
import keras.backend as K
import tensorflow as tf
from tensorflow.keras import optimizers, layers
from tensorflow.keras import losses
from tensorflow.keras import callbacks
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Model
import pickle
import time
import csv
from functools import partial
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import scipy.stats as stats
import tensorflow_addons as tfa
import seaborn as sns
import scipy
import warnings
import rasterio
import math
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()
# warnings.filterwarnings("ignore")
# tf.autograph.set_verbosity(1)
import sys

import config
###priv imports
import config as cfg
import visualizations
import nn_utils as nnu
import nn_models as nnm
import helper_utils as hu
import water_w_regression_methods as wrm
# import geo_utils as gu

In [ ]:
#####This needs to be done before using tensorflow or keras (e.g. importing losses in config)
###Some general information and settings
# print some general information
print('keras v', keras.__version__)
print('tf keras v', tf.keras.__version__)
print('tf v', tf.__version__)
# to do: try non eager execution graph?
print('tf eager execution', tf.executing_eagerly())
# force channels-first ordering
K.set_image_data_format('channels_first')
print('Image data format', K.image_data_format())

#turn off cryptic warnings, Note that you might miss important warnings! If unexpected stuff is happening turn it on!
#https://github.com/tensorflow/tensorflow/issues/27023
#Thanks @Mrs Przibylla
#'1' = Infos, '2' = warnings, '3' = Errors
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# do not assign complete gpu-memory but grow it as needed
# allows to run multiple models at once (otherwise whole gpu memory gets allocated/gpu gets blocked)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

###Define Model and MirroredStrategy (for mulit-gpu usage) - shall be initiated at the beginning of the program
# Create a MirroredStrategy and pass the GPUs
gpus = ["GPU:" + str(i) for i in cfg.gpus]
# https://keras.io/guides/distributed_training/
print('gpus', gpus)
strategy = tf.distribute.MirroredStrategy(gpus)
print('Number of GPUs MirroredStrategy: {}'.format(strategy.num_replicas_in_sync))